In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [14]:
chromedriver_location = "chrome_driver/chromedriver"

In [15]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
#seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [16]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [42]:
main_df = pd.DataFrame({
    "date" : [],
    "case_num" : [],
    "case_style" : [],
    "time": [],
    "days": [],
    "event": [],
    "location": [],
    "judge": []
})

In [21]:
date_range = ['8/16/2021', '8/17/2021', '8/18/2021', '8/19/2021', '8/20/2021', '8/21/2021', '8/22/2021']

In [43]:
# FORM INPUT AND SUBMIT

# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')
# 7 day search results
#driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")


for date in date_range:
    # clear the date text box
    driver.find_element_by_xpath(date_field).clear()
    # store the district dropdown in an object using Select
    courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
    courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles
    # date
    driver.find_element_by_xpath(date_field).send_keys(date)
    # without the pause, the inputs are too fast for casenet to handle
    time.sleep(1) 
    # submit 
    driver.find_element_by_xpath(submit_button).click()

    if len(driver.find_elements_by_xpath('//*[contains(text(), "Your query returned no matches to be viewed at this site.")]')) > 0:
        continue
    # INFO GATHERING
    i=1
    data = []
    raw_text = []
    # get current date
    current_date = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]').text
    # collect information from the table
    while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
        data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
        i += 1
    # parse text into a list
    for d in data:
        for i in d:
            raw_text.append(i.text)       
    # slices the data into lists of each feild 
    organized_data = slice_per(raw_text, 9)
    # create data frame to hold the data
    df = pd.DataFrame({
        "date" : current_date,
        "case_num" : organized_data[0],
        "case_style" : organized_data[1],
        "time": organized_data[2],
        "days": organized_data[3],
        "event": organized_data[5],
        "location": organized_data[7],
        "judge": organized_data[8]
    })
    # add collected data to main dataframe
    main_df = pd.concat([df, main_df], axis=0, join='inner')

    # go back to previous page
    driver.back()


In [45]:
len(main_df)

13

In [44]:
main_df

,date,case_num,case_style,time,days,event,location,judge
0,"Thursday, August 19, 2021",1911-CR01411,ST V BRITTANY N ROSE,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
1,"Thursday, August 19, 2021",2011-CR04156,ST V PERCY LEE BENNETT,1:30 PM,1 OF 1,Disposition Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
2,"Thursday, August 19, 2021",1911-CR03589,ST V BRITTANY NICOLE ROSE,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
0,"Wednesday, August 18, 2021",1611-CR03626-01,ST V NICOLE MARIE MILLER-POLKINGHORNE,9:00 AM,1 OF 1,Bond Appearance Hearing,Location: DIVISION 2 COURTROOM St Charles Circ...,Judge/Commissioner: DEBORAH JEAN ALESSI
1,"Wednesday, August 18, 2021",2011-CR00837,ST V DYLAN T DEPTULA,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 9 COURTROOM St Charles Circ...,Judge/Commissioner: ELIZABETH WALKER SWANN
2,"Wednesday, August 18, 2021",1911-CR04606,ST V HARRY BERNARD SCHIEN IV,9:00 AM,1 OF 1,Counsel Status Hearing,Location: DIVISION 9 COURTROOM St Charles Circ...,Judge/Commissioner: ELIZABETH WALKER SWANN
0,"Monday, August 16, 2021",1911-CR01190-01,ST V JEFFREY CALVIN HILL,8:30 AM,1 OF 1,Plea Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
1,"Monday, August 16, 2021",1911-CR01457-01,ST V SEAN D. GWIN,8:30 AM,1 OF 1,Disposition Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
2,"Monday, August 16, 2021",1811-CR03256-01,ST V JULIANNE MARIE POPE,9:00 AM,1 OF 1,Sentencing Hearing,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
3,"Monday, August 16, 2021",1911-CR02239-01,ST V BRIAN KEITH BILLINGSLEY,1:30 PM,1 OF 1,Pre-trial Conference,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
